In [50]:
import sys, os
sys.path.append(os.path.abspath('..'))
import numpy as np
from ImageLab import *
import cv2
from PIL import Image

Create a function that converts an image to hough's space.

In [51]:
def Hough_Space(image):
    image = image.astype(np.float32)
    
    # Define the Hough space
    theta = np.linspace(-np.pi / 2, np.pi / 2, 180)
    width, height = image.shape[:2]
    diag_len = np.sqrt(width * width + height * height)
    diag_len = int(diag_len)
    rho = np.linspace(-diag_len, diag_len, diag_len * 2)

    # Compute the Hough space
    hough_space = np.zeros((len(rho), len(theta)))
    for x in range(width):
        for y in range(height):
            if image[x][y] != 0:
                for t_idx in range(len(theta)):
                    t = theta[t_idx]
                    r = x * np.cos(t) + y * np.sin(t)
                    r_idx = np.argmin(np.abs(rho - r))
                    hough_space[r_idx][t_idx] += 1

    return hough_space

In [52]:
# Import the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

# Define a function that takes a list of points and returns the labels and the number of clusters
def dbscan_cluster(points):
  # Convert the points to a numpy array
  points = np.array(points)
  # Create a dbscan object with some parameters
  dbscan = DBSCAN(eps=0.5, min_samples=5)
  # Fit the dbscan object to the points and get the labels
  labels = dbscan.fit_predict(points)
  # Get the number of clusters (excluding noise)
  n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
  # Return the labels and the number of clusters
  return labels, n_clusters


Create a function to convert houghs points back to normal space.

In [65]:
import numpy as np
import math

def hough_lines(image, threshold, folder, name):
  # image is a binary array with 0s indicating non-edges and 1s indicating edges
  # returns a list of tuples (rho, theta) representing lines in polar space
  
  # get the dimensions of the image
  height, width = image.shape
  
  # get the maximum possible distance from the origin to any point in the image
  max_dist = int(math.sqrt(height**2 + width**2))
  
  # create an array of theta values from -90 to 90 degrees
  theta = np.deg2rad(np.arange(-90, 90))
  
  # create an array of rho values from -max_dist to max_dist
  rho = np.linspace(-max_dist, max_dist, 2 * max_dist)
  
  # create an empty accumulator array with dimensions (rho, theta)
  accumulator = np.zeros((2 * max_dist, len(theta)))
  
  # find the indices of the edge pixels in the image
  y_indices, x_indices = np.nonzero(image)
  
  # loop through each pixel in the image
  for x in range(width):
    for y in range(height):
      # if the pixel is an edge point
      if image[y, x] > 0:
        # calculate the rho values for each theta value
        rhos = x * np.cos(theta) + y * np.sin(theta)
        # round the rho values to the nearest integer
        rhos = np.round(rhos).astype(int)
        for i in range(len(rhos)):
          # increment the corresponding cell in the accumulator array
          accumulator[int(rhos[i] + max_dist), int(np.rad2deg(theta[i])) + 90] += 1
  
  
  stretched_houghs = ImageFilters(accumulator).process(contrast_stretch())
  segmented_houghs = ImageSegment(stretched_houghs).process(Global_Threshold(100))
  
  segmented = segmented_houghs * stretched_houghs
  
  _=ImageUtil(segmented).save_image_to_folder('accumulator', name)
  
  # create an empty list to store the lines
  line_points = []
  
  # loop through each cell in the accumulator array
  for r in range(2 * max_dist):
    for t in range(len(theta)):
      rh = rho[r]
      th = theta[t]
      # append the line to the list as a tuple
      line_points.append((rh, th))

In [66]:
import matplotlib.pyplot as plt

def plot_lines_on_image(image, lines):
  # image is a numpy array representing an image
  # lines is a list of tuples (x1, y1, x2, y2) representing the endpoints of each line
  # plots the image and the lines on a figure and shows it
  
  # create a figure and an axes object
  fig, ax = plt.subplots()
  
  # plot the image on the axes object
  ax.imshow(image)
  
  # loop through each line in the list
  for line in lines:
    # unpack the endpoints of the line
    x1, y1, x2, y2 = line
    # plot the line on the axes object
    ax.plot([x1, x2], [y1, y2], color="red", linewidth=1)
  
  # show the figure
  plt.show()
  
  return fig

In [67]:
kernel = np.array([[0, 1, 0],
                  [1, 1, 1],
                  [0, 1, 0]])


def pipeline(img_path, folder, name):
    image = Image.open(img_path).convert('L')
    image = np.array(image)
    # inverted = ImageColorSpace(image).process(inversion())
    # print(np.max(inverted))
    print(image.shape)
    dilated = ImageProcessor(image).process(Dilation(), kernel)
    closed = ImageProcessor(dilated).process(Erosion(), kernel)
    sauvola = ImageSegment(closed).process(Pixel_Filter(3, 50, 'Sauvola'))
    # Apply edge detection
    # edges = cv2.Canny(closed,50,150,apertureSize = 3)
    seggrey = sauvola * closed
    seggrey = np.squeeze(seggrey)
    
    print(seggrey.shape)
    # _=ImageUtil(seggrey).save_image_to_folder('folder1', name)
    
    hough_lines(seggrey, 20, folder, name)
    
    # stretched_houghs = ImageFilters(houghs).process(contrast_stretch())
    # thresholded = ImageSegment(stretched_houghs).process(Global_Threshold(100))
    return seggrey

In [68]:
# set the path to your test images folder
target_folder = 'test_images'
output_folder = 'detection_images'

chi_difference = 1

# loop through each image file in the folder
for file_name in os.listdir(target_folder):
    # check if the file is an image file
    if file_name.endswith('.jpg') or file_name.endswith('.png'):
        
        img_path = os.path.join(target_folder, file_name)
        file_name = os.path.splitext(file_name)[0]
        
        segimg = pipeline(img_path, 'Segmented', f'{file_name}.png')
                

(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
